# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

### Import libraries and load Starbucks data

In [ ]:
# standard libraries
import pandas as pd
import numpy as np
import math, json , re, os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, HTML, display
from tqdm import tqdm
from progressbar import progressbar

# ML specific classes through sklearn
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# jupyter magic to display plots directly in the notebook
%matplotlib inline

# use vector graphics format for nicer plots
%config Inline.Backend.figure_format = 'svg'

plt.style.use('seaborn')

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# 1. Exploratory Analysis

#### 1.1 Lets look at the data and check out inconsistencies

## 1. Profile Dataset

In [ ]:
display(profile.head(), profile.isna().sum().to_frame().rename(columns={0:'Null count'}))

In [ ]:
profile.query("age == 118")

#### We find there are null values in Gender (along with'O') and Income columns. As we can see above, age is 118 for all rows where null value is present in gender and income columns

In [ ]:
# let's first rename id to customer_id for more clarity
profile = profile.rename(columns={'id':'customer_id'})

# No. of users 
print('profile size: {}'.format(profile['customer_id'].unique().shape[0]))

# gender distribution
display(profile['gender'].value_counts().to_frame())

# income distribution by gender 
plt.figure(figsize=(10,6))
sns.histplot(data=profile, x="income", hue="gender", kde=True);
plt.tight_layout()
# plt.savefig('plots/profile_income_dist.svg')
plt.show()

# age distribution by gender
plt.figure(figsize=(10,6))
sns.histplot(data=profile, x="age", hue="gender", kde=True);
plt.tight_layout()
# plt.savefig('plots/profile_age_dist.svg')
plt.show()

In [ ]:
# Convert "became_member_on" to standard python datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format = '%Y%m%d')
profile['became_member_in'] = profile['became_member_on'].apply(lambda x: x.year)
# Compute how long the user has been a member of the Starbucks app - let's say the anchor point is say 30-Sep-2020
profile['member_since_in_months'] = (pd.to_datetime("30-Sep-2020") - profile['became_member_on']).astype('timedelta64[M]').astype('int')

profile.head()

In [ ]:
# "became member in" distribution by gender - how many customers are new and how many are long term members?
members_by_year = profile.groupby('became_member_in')['customer_id'].count().to_frame().rename(columns={'customer_id':'no_of_customers'})

plt.figure(figsize=(10,6))
sns.barplot(data = members_by_year, x=members_by_year.index, y='no_of_customers')
plt.tight_layout()
# plt.savefig('plots/profile_member_joined_dist.svg')
plt.show()

In [ ]:
# "loyalty" distribution by gender
plt.figure(figsize=(10,6))
sns.histplot(data=profile, x="member_since_in_months", kde=True);
plt.tight_layout()
# plt.savefig('plots/profile_loyalty_dist.svg')
plt.show()

## 2. Portfolio Dataset

In [ ]:
display(portfolio.head(), portfolio.isna().sum().to_frame().rename(columns={0:'Null count'}))

In [ ]:
# let's first rename id to offer_id for more clarity
portfolio = portfolio.rename(columns={'id':'offer_id'})

# let's check the number of outstanding offers
portfolio_stats = portfolio.groupby('offer_type')['offer_id'].count().to_frame()
portfolio_stats.loc["total"] = portfolio_stats.sum(axis=0)
portfolio_stats

In [ ]:
# we can see multiple channels through which offers have been delivered to users - let's check them
portfolio['channels'].explode().unique().tolist()

In [ ]:
# we also see that each offer type has a reward and difficulty columns - would be interesting to see if you get more reward for spending more? :)
plt.figure(figsize=(10,6))
sns.scatterplot(data=portfolio, x="difficulty", y="reward", hue="offer_type")
plt.tight_layout()
# plt.savefig('plots/portfolio_reward_vs_difficulty.svg')
plt.show()

## 3. Transcript Dataset

In [ ]:
display(transcript.head(), transcript.isna().sum().to_frame().rename(columns={0:'Null count'}))

In [ ]:
# let's first rename person to customer_id for more clarity
transcript = transcript.rename(columns={'person':'customer_id'})

In [ ]:
# No. of users 
print('transcript size: {}'.format(transcript['customer_id'].unique().shape[0]))

#### It looks as though the number of people in transcript are the same as the number of people in the profile dataset, so that is good news.

In [ ]:
# what are the different types of values - we're mainly interested in the dictionary keys
dict_keys = [list(value.keys()) for value in transcript['value']]
# need to flatten list of lists that may be present - occurs when a dictionary has multiple keys
set([item for sublist in dict_keys for item in sublist])

In [ ]:
# event distribution
event_dist = transcript.groupby('event')['customer_id'].count().to_frame()
event_dist = event_dist.div(event_dist.sum(axis=0), axis=1).multiply(100)
ax = event_dist.rename(columns={'customer_id':'pct'}).sort_values(by='pct', ascending=True).plot(kind='barh', figsize=(10,6), legend=None);
plt.title('Events distribution', fontsize=14)
plt.xlabel('percentage of total events')
plt.tight_layout()
# plt.savefig('plots/transcript_event_dist.svg', dpi=300)
plt.show()

In [ ]:
# let's expand values column and get keys as column headers and get rid of the dictionary
transcript = pd.concat([transcript.drop(['value'], axis=1), transcript['value'].apply(pd.Series)], axis=1)
# there are duplicate offer id and offer_id columns - need to clean them up
transcript['offer_id'] = transcript['offer_id'].fillna(transcript['offer id'])
transcript = transcript.drop(columns=['offer id'])
transcript.head()

In [ ]:
# check if no. of users in transcript is the same as in the profile dataset
users_in_transcript = list(transcript['customer_id'].unique())
users_in_profile = list(profile['customer_id'].unique())

in_transcript_but_not_in_profile  = [x for x in users_in_transcript if x not in set(users_in_profile)]
in_profile_but_not_in_transcript  = [x for x in users_in_profile if x not in set(users_in_transcript)]

print('Number of users in transcript but not in profile dataset: '+str(len(in_transcript_but_not_in_profile)))
print('Number of users in profile but not in transcript dataset: '+str(len(in_profile_but_not_in_transcript)))

#### Create a temporary dataset to analyze events and event_type to see which offers do the users prefer

In [ ]:
temp = transcript.copy()
temp = temp.merge(portfolio[['offer_id','offer_type']], how='left', on='offer_id')
temp.head()

In [ ]:
temp_group = temp.groupby(['event','offer_type'])['event'].count().to_frame().rename(columns={'event':'count'})

# get percentages event
temp_group['percentage'] = temp_group.div(temp_group.sum(level=0), level=0)
temp_group['percentage'].unstack(level=0).plot(kind='barh', subplots=True, figsize=(10,8));
plt.tight_layout()
# plt.savefig('plots/transcript_portfolio_offer_preference.svg', dpi=300)
plt.show()

# 2. Data cleaning and preprocessing for machine learning algorithms

#### Now lets clean up all three dataframes into a usable format and join them in one single dataframe

### 2.1 Clean profile dataset 

In [ ]:
def clean_preprocess_profile(profile):
    """
    Takes the profile dataframe and cleans it by creating one-hot encodings as well as handling null values

    """
    # We can safely remove null values since they form a very small part of the dataset
    # Remove customers with null income data
    profile = profile.dropna(subset=['gender','income'])
    profile = profile[profile['gender'] != 'O']

    # Specifying age range
    min_age = np.int(np.floor(np.min(profile['age'])/10)*10)
    max_age = np.int(np.ceil(np.max(profile['age'])/10)*10)
    age_bins = range(min_age, max_age + 10, 10)
    
    profile['age_range'] = pd.cut(x=profile['age'], bins=age_bins, right=True).astype('str')
    # One-hot encode age_range column
    age_dummies = pd.get_dummies(profile['age_range'])
    
    # Specifying income range and one hot encoding
    min_income = np.floor(np.min(profile['income']))
    max_income = np.ceil(np.max(profile['income']))
    income_bins = np.linspace(min_income, max_income, 10, endpoint=True)
    
    profile['income_range'] = pd.cut(x=profile['income'], bins=income_bins, right=True, include_lowest=True).astype('str')
    # One-hot encode income_range column
    income_dummies = pd.get_dummies(profile['income_range'])
    
    # Finally, transform gender from a character to a number using LabelBinarizer
    convert_to_binary = LabelBinarizer()
    profile['gender'] = convert_to_binary.fit_transform(profile['gender'])
    
    gender_map = {}
    for gender_int in convert_to_binary.classes_:
        gender_map[gender_int] = convert_to_binary.transform([gender_int])[0,0]
    
    # the year that a customer became a member in is not uniformly distributed (6th cell) - potential customer differentiator
    # let's add dummies for that as well
    year_dummies = pd.get_dummies(profile['became_member_in'])
    
    # concat the dummies to the profile dataset and drop the original columns
    profile = (pd.concat([profile, age_dummies, income_dummies, year_dummies], axis=1, sort=False)
                .drop(columns=['age','age_range','income','income_range',
                               'became_member_on','became_member_in','member_since_in_months']))
    
    return profile, gender_map

In [ ]:
cleaned_profile, gender_map = clean_preprocess_profile(profile)
print(gender_map)
cleaned_profile.head()

In [ ]:
cleaned_profile.columns

### 2.2 Clean portfolio dataset 

In [ ]:
def clean_preprocess_portfolio(portfolio):
    """
    Takes the portfolio dataframe and cleans it by creating one-hot encodings
    
    """
    # No null handling required since there are no NULL values
    # One-hot encode channels column - using sckit-learn module
    multi_label_binary = MultiLabelBinarizer()
    multi_label_binary.fit(portfolio['channels'])
    
    channels_dummies = pd.DataFrame(multi_label_binary.transform(portfolio['channels']), columns=multi_label_binary.classes_)
    
    # One-hot encode offer_type column
    offer_type_dummies = pd.get_dummies(portfolio['offer_type'])
    
    # that's it - concat now
    portfolio = pd.concat([portfolio, channels_dummies, offer_type_dummies], axis=1, sort=False) \
                  .drop(columns=['offer_type', 'channels'])
    
    return portfolio

In [ ]:
cleaned_portfolio = clean_preprocess_portfolio(portfolio)
cleaned_portfolio.head()

### 2.3 Clean transcript dataset 

In [ ]:
def clean_preprocess_transcript(transcript):
    """ 
    Transforms the transcript dataframe and clean it by creating one-hot encodings 
    Also split the dataframe into seperate offers and transaction datasets
    
    """
    # no need to clean for customers not in profile since both have the same users - cell 18
    # Convert time column from hours to days
    transcript['time'] = transcript['time'] / 24.0
    
    # let's first get dummies for events
    event_dummies = pd.get_dummies(transcript['event'])
    event_dummies.columns = [col.replace(' ','_') for col in event_dummies.columns]

    # now concat to get final dataframe
    cleaned_transcript = pd.concat([transcript, event_dummies], axis=1, sort=False).drop(columns=['event'])
    
    offer_cols = ['customer_id','offer_id','time','offer_completed','offer_received','offer_viewed']
    transac_cols = ['customer_id','time','amount','reward']
    cleaned_offer_dataset = cleaned_transcript.query("offer_completed==1 or offer_received==1 or offer_viewed==1")[offer_cols]
    cleaned_transaction_dataset = cleaned_transcript.query("transaction==1")[transac_cols]
    
    return cleaned_transcript, cleaned_offer_dataset, cleaned_transaction_dataset

In [ ]:
cleaned_transcript, cleaned_offer_dataset, cleaned_transaction_dataset = clean_preprocess_transcript(transcript)
cleaned_transcript.head()

In [ ]:
cleaned_offer_dataset.head()

In [ ]:
cleaned_transaction_dataset.head()

## Obersevations
#### What is an effective offer?
1. For BOGO and discount offers, an effective offer would be defined if the events are defined in this chronological order:
   **offer received > offer viewed > transaction > offer completed**
2. For Informational offer, the event chronology would be:
   **offer received > offer viewed > transaction**
   
#### What is not counted as an effective offer?
1. User recieved and viewed the offer but did not transact: **offer received > offer viewed** (no offer completed/transaction events)
2. User received offer but did not do anything: **offer received** (did not even view it)
3. Users who transacted w/o receiving an offer, after the offer is completed, before the offer is received or before the the offer is viewed. The different timelines can be summarized as:
    - **transaction**
    - **offer received > "do nothing"** (did not even view the offer)
    - **offer received > transaction > offer completed > offer viewed**
    - **transaction > offer received > offer completed > offer viewed**
    - **offer received > transaction > offer viewed > offer completed**
    - **offer received > transaction** (only applicable to informational offer)
    - **offer received > transaction > offer viewed** (only applicable to informational offer)
    
(3) can be summarized as - if an offer completed or transaction event occurs before an offer viewed event occurs.

### Let's sort our dataframe by customer_id and time and iterate through each customer id to label the dataset (computationally intensive)

In [ ]:
def construct_effective_offer_label(profile, portfolio, offers, transactions):
    # define the unique customer_ids to loop over
    customer_ids = offers['customer_id'].unique().tolist()
    
    all_offers_labeled = []
    for _user in tqdm(range(len(customer_ids))):
        events = ['offer_completed','offer_received','offer_viewed']
        
        # get customer_id from the list
        user_id = customer_ids[_user]
        
        # get profile data for the user
        user_profile = profile.query("customer_id == @user_id").drop(columns='customer_id')
        # get offer data for the user
        user_offers_data = offers.query("customer_id == @user_id").drop(columns='customer_id')
        # get transaction data for the user
        user_transactions_data = transactions.query("customer_id == @user_id").drop(columns='customer_id')
        
        offer_received = offers.query("offer_received == 1").drop(columns=events)
        offer_viewed = offers.query("offer_viewed == 1").drop(columns=events)
        offer_completed = offers.query("offer_completed == 1").drop(columns=events)
        
        # loop over each offer for the particular user
        all_offers_user = []
        for offer in range(len(offer_received)):
            # fetch offer id for the offer 
            offer_id = offer_received.iloc[offer]['offer_id']
            
            # extract offer data from portfolio
            offer_row = portfolio.query("offer_id == @offer_id").drop(columns=['offer_id'])
            
            # extract offer duration from offer row
            offer_duration = offer_row['duration'].values[0]

            # compute offer start and offer end times (in days)
            start_offer = offer_received.iloc[offer]['time']
            end_offer = start_offer + offer_duration
            
            # check if offer was viewed/completed and was transcated for the duration of the offer
            offer_transactions = user_transactions_data.query("time >= @start_offer and time <= @end_offer")
            
            offer_viewed_mask = (offer_viewed['time'] >= start_offer) & (offer_viewed['time'] <= end_offer)
            offer_completed_mask = (offer_completed['time'] >= start_offer) & (offer_completed['time'] <= end_offer)
            offer_successful_mask = (offer_viewed_mask.sum() > 0) & (offer_completed_mask.sum() > 0)
            
            # make a dictionary that describes the current customer offer
            offer_summary = {'effective_offer': int(offer_successful_mask), 'offer_id': offer_id, 'customer_id': user_id, 
                             'time': start_offer, 'total_amount_transacted_for_offer': offer_transactions['amount'].sum()}
            
            offer_summary_df = pd.DataFrame([offer_summary])
            # prepare to concat the features now from offer_row and user_offers_data
            offer_labeled = (pd.concat([offer_summary_df, offer_row, user_profile], axis=1, sort=False))
            
            all_offers_user.append(offer_labeled)
        
        # aggregate all offers for the user
        user_offers_labeled = pd.concat(all_offers_user)        
        
        all_offers_labeled.append(user_offers_labeled)
    
    # aggregate for all users/customer_ids
    all_offers_labeled_df = pd.concat(all_offers_labeled)
    
    return all_offers_labeled_df

In [ ]:
%%time
effective_offer_labeled_dataset = (construct_effective_offer_label(cleaned_profile, 
                                                                   cleaned_portfolio, 
                                                                   cleaned_offer_dataset, 
                                                                   cleaned_transaction_dataset))

In [ ]:
effective_offer_labeled_dataset.head()

In [ ]:
effective_offer_labeled_dataset.to_pickle('effective_offer_labeled_dataset.pkl')